In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

In [3]:
A = []
B = []
C = []
table = soup.find('table', class_="wikitable sortable")

In [4]:
for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) > 0: #this excludes the column titles which have no td values
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [5]:
df = pd.DataFrame(A , columns = ['PostalCode'])
df['Borough'] = B
df['Neighborhood'] = C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df2 = df[df['Borough'] != 'Not assigned']
df2.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
df3 = df2.groupby(['PostalCode', 'Borough']).agg(lambda col: ','.join(col))
df3.head()

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood\n,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [8]:
df3["Neighborhood"]= df3["Neighborhood"].replace('\n', '', regex=True)

In [9]:
for index, row in df3.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = index[1]

In [10]:
df4 = df3.reset_index()
df4
df4.shape

(103, 3)

In [11]:
geo_url="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(geo_url)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
lat = []
long = []
for index, row in df4.iterrows():
    value = str(row['PostalCode'])
    geo_data1 = geo_data[geo_data['Postal Code'] == value]
    Latitude = float(geo_data1['Latitude'])
    Longitude = float(geo_data1['Longitude'])
    
    lat.append(Latitude)
    long.append(Longitude)

In [13]:
df4['Latitude'] = lat
df4['Longitude'] = long
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
#!conda install -c conda-forge folium=0.5.0  
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

In [15]:
map_Toronto = folium.Map(location=[43.6532,-79.3832], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
map_Toronto

In [16]:
df5 = df4['Borough'].unique()
df5
df5_list = df5.tolist()

In [17]:
Borough_Code = []
for borough in df4['Borough']:
    Borough_Code.append(df5_list.index(borough))

In [18]:
df4['Borough Code'] = Borough_Code
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Borough Code
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


In [19]:
colors_array = cm.rainbow(np.linspace(0, len(df5) - 1))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [20]:
map_Toronto2 = folium.Map(location=[43.6532,-79.3832], zoom_start=10)
for lat, lng, borough, neighborhood, borough_code in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood'], df4['Borough Code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[borough_code-1],
        fill=True,
        fill_color=rainbow[borough_code-1],
        fill_opacity=0.7).add_to(map_Toronto2)  
map_Toronto2